In [10]:
# STEP 1: Install required libraries
!pip install geopandas simplekml pyshp

In [13]:
# STEP 2: Upload multiple files (all parts of shapefiles)
from google.colab import files
import zipfile
import os

uploaded = files.upload()  # Upload all .shp, .shx, .dbf, .prj, etc.

Saving Bajhang_r.shx to Bajhang_r.shx
Saving Bajura.shx to Bajura.shx
Saving Bara.shx to Bara.shx
Saving Dadeldhura_r.shx to Dadeldhura_r.shx
Saving Dailekh.shx to Dailekh.shx
Saving Dhanusha.shx to Dhanusha.shx
Saving Dolpa.shx to Dolpa.shx
Saving Doti_r.shx to Doti_r.shx
Saving Jajarkot.shx to Jajarkot.shx
Saving Jumla.shx to Jumla.shx
Saving Kailali.shx to Kailali.shx
Saving Mugu_R.shx to Mugu_R.shx
Saving Rautahat.shx to Rautahat.shx
Saving Rukum_west.shx to Rukum_west.shx
Saving Saptari.shx to Saptari.shx
Saving Sarlahi.shx to Sarlahi.shx
Saving Siraha.shx to Siraha.shx
Saving Surkhet.shx to Surkhet.shx
Saving Bajhang_r.shp to Bajhang_r.shp
Saving Bajura.shp to Bajura.shp
Saving Bara.shp to Bara.shp
Saving Dadeldhura_r.shp to Dadeldhura_r.shp
Saving Dailekh.shp to Dailekh.shp
Saving Dhanusha.shp to Dhanusha.shp
Saving Dolpa.shp to Dolpa.shp
Saving Doti_r.shp to Doti_r.shp
Saving Jajarkot.shp to Jajarkot.shp
Saving Jumla.shp to Jumla.shp
Saving Kailali.shp to Kailali.shp
Saving Mug

In [14]:

# STEP 3: Extract all uploaded files into a folder
input_folder = "/content/shapefiles"
os.makedirs(input_folder, exist_ok=True)

for fname in uploaded.keys():
    os.rename(fname, os.path.join(input_folder, fname))

# STEP 4: Convert all SHP to KML
import geopandas as gpd
import simplekml

def convert_shp_folder_to_kml_zip(input_folder, zip_output_path):
    kml_output_folder = os.path.join(input_folder, "kml_outputs")
    os.makedirs(kml_output_folder, exist_ok=True)

    for file in os.listdir(input_folder):
        if file.endswith(".shp"):
            shp_path = os.path.join(input_folder, file)
            name = os.path.splitext(file)[0]
            kml_path = os.path.join(kml_output_folder, f"{name}.kml")
            convert_single_shp_to_kml(shp_path, kml_path)

    # Zip all KMLs
    with zipfile.ZipFile(zip_output_path, 'w') as zipf:
        for file in os.listdir(kml_output_folder):
            zipf.write(os.path.join(kml_output_folder, file), arcname=file)

    print(f"KMLs zipped at: {zip_output_path}")

def convert_single_shp_to_kml(shapefile_path, output_kml_path):
    gdf = gpd.read_file(shapefile_path)
    kml = simplekml.Kml()

    for _, row in gdf.iterrows():
        geom = row['geometry']
        kml_geom = geom.__geo_interface__
        geometry_type = kml_geom['type']
        field_name = gdf.columns[0]
        placemark = kml.newmultigeometry(name=str(row[field_name]))

        if geometry_type == 'Point':
            placemark.newpoint(coords=[kml_geom['coordinates']])
        elif geometry_type == 'LineString':
            line = placemark.newlinestring()
            line.coords = kml_geom['coordinates']
        elif geometry_type == 'Polygon':
            poly = placemark.newpolygon()
            poly.outerboundaryis = kml_geom['coordinates'][0]

        for field in gdf.columns:
            if field != 'geometry':
                placemark.extendeddata.schemadata.newsimpledata(field, str(row[field]))

    kml.save(output_kml_path)

# STEP 5: Run the conversion and zip output
zip_output_path = "/content/kml_outputs.zip"
convert_shp_folder_to_kml_zip(input_folder, zip_output_path)

# STEP 6: Provide download button
files.download(zip_output_path)


KMLs zipped at: /content/kml_outputs.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [16]:
# STEP 1: Install libraries
!pip install geopandas simplekml pyshp

# STEP 2: Upload all shapefile parts
from google.colab import files
import os

uploaded = files.upload()  # Upload all .shp, .shx, .dbf, .prj etc.

# Create working folder
input_folder = "/content/shapefiles"
os.makedirs(input_folder, exist_ok=True)

# Move uploaded files
for fname in uploaded.keys():
    os.rename(fname, os.path.join(input_folder, fname))

# STEP 3: Convert each polyline to individual KML files
import geopandas as gpd
import simplekml
import zipfile

output_kml_folder = "/content/kml_outputs"
os.makedirs(output_kml_folder, exist_ok=True)

def export_individual_polylines(shapefile_path, output_dir):
    gdf = gpd.read_file(shapefile_path)
    lines_gdf = gdf[gdf.geometry.type.isin(['LineString', 'MultiLineString'])]

    if lines_gdf.empty:
        print(f"No polylines in {shapefile_path}")
        return

    base_name = os.path.splitext(os.path.basename(shapefile_path))[0]

    for idx, row in lines_gdf.iterrows():
        kml = simplekml.Kml()
        geom = row.geometry
        coords = []

        if geom.geom_type == 'LineString':
            coords = list(geom.coords)
        elif geom.geom_type == 'MultiLineString':
            for part in geom.geoms:  # ✅ FIX HERE
                coords.extend(part.coords)

        line_name = f"{base_name}_{idx+1}"
        linestring = kml.newlinestring(name=line_name)
        linestring.coords = coords

        for field in lines_gdf.columns:
            if field != 'geometry':
                linestring.extendeddata.schemadata.newsimpledata(field, str(row[field]))

        kml_path = os.path.join(output_dir, f"{line_name}.kml")
        kml.save(kml_path)

# STEP 4: Process all .shp files in input folder
for file in os.listdir(input_folder):
    if file.endswith(".shp"):
        shp_path = os.path.join(input_folder, file)
        export_individual_polylines(shp_path, output_kml_folder)

# STEP 5: Zip all individual KMLs
zip_path = "/content/polylines_kmls.zip"
with zipfile.ZipFile(zip_path, 'w') as zipf:
    for file in os.listdir(output_kml_folder):
        zipf.write(os.path.join(output_kml_folder, file), arcname=file)

# STEP 6: Download zipped KMLs
files.download(zip_path)


Saving Bajhang_r.dbf to Bajhang_r.dbf
Saving Bajhang_r.prj to Bajhang_r.prj
Saving Bajhang_r.sbn to Bajhang_r.sbn
Saving Bajhang_r.sbx to Bajhang_r.sbx
Saving Bajhang_r.shp to Bajhang_r.shp
Saving Bajhang_r.shp.xml to Bajhang_r.shp.xml
Saving Bajhang_r.shx to Bajhang_r.shx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>